# ambr250 Data Export

Specify desired parameters in the 'export_parameters.txt' file.

Go to Cell > Run All to export .xlsx files for HT1-HT12.

In [16]:
import pandas as pd
import os
import re
import json

In [17]:
# Define filepaths
project_name = 'ABFr_ambr_03+ABFpj__02'
data_directory = project_name + '/Data/00001/B/'
export_params_file = 'export_parameters.txt'
library_file = "param_library.json"

In [18]:
def write_to_csv(filename, patterns):
    '''
    Converts input .csv files into pandas dataframe, selects for export parameters
    and writes as compiled .csv file.
    
    string filename: filepath to bioreactor (HT1-HT12) that is being exported
    dictionary patterns: dictionary with pattern names and their corresponding regex patterns
    '''
    reactor_data = os.path.join(data_directory, filename)
    worksheet_name = 'HT' + str(int(filename))
    compiled = pd.DataFrame({'' : []})
    # Iterate through export parameters
    for parameter in open(export_params_file):
        # Get pattern for export parameter from library
        pattern = patterns[parameter.rstrip('\n')]
        # Iterate through .csv files and check for pattern match
        for sheet_name in os.listdir(reactor_data):
            if re.match(pattern, sheet_name):
                sheet_path = os.path.join(reactor_data, sheet_name)
                df = pd.read_csv(sheet_path, usecols=[0,1], header=2)
                # Reformat column header
                df.rename(columns={"VariableKey": "Time"}, inplace=True)
                # Assign compiled dataframe or concatenate to existing dataframe
                if compiled.empty:
                    compiled = df
                else:
                    compiled = pd.concat([compiled, df], axis=1)

    # Export completed dataframe as .csv file
    export_filename = project_name + '_export_data_' + worksheet_name + '.csv'
    compiled.to_csv(export_filename)  
    print("Successfully exported data from " + str(worksheet_name))
            
def compile_bioreactors(directory):
    '''
    Iterates through HT1-HT12 and calls export function
    
    string directory: filepath to directory containing HT1-HT12 data.
    '''
    with open(library_file) as param_library:
        # Load parameter pattern library
        param_patterns = json.load(param_library)
    for index, filename in enumerate(os.listdir(directory)):
            # Match for bioreactor name (HT1-HT12)
            if re.match('\d{2}', filename):
                write_to_csv(filename, param_patterns)
    print("All data exported.")

In [19]:
compile_bioreactors(data_directory)

Successfully exported data from HT3
Successfully exported data from HT4
Successfully exported data from HT5
Successfully exported data from HT2
Successfully exported data from HT11
Successfully exported data from HT10
Successfully exported data from HT7
Successfully exported data from HT9
Successfully exported data from HT8
Successfully exported data from HT1
Successfully exported data from HT6
Successfully exported data from HT12
All data exported.
